# 서울시 상권 및 생활인구 데이터 다운로드

In [ ]:
import requests
import json
import pandas as pd


def create_api_url(key: str, service_id: str, *args) -> str:
    url = f"http://openapi.seoul.go.kr:8088/{key}/json/{service_id}"
    for arg in args:
        url += f"/{arg}"
    return url


def create_url(
    key, service_id, start_index, end_index, year_code=None, format="json"
) -> str:
    url = f"http://openapi.seoul.go.kr:8088/{key}/{format}/{service_id}/{start_index}/{end_index}"
    if year_code is not None:
        url += f"/{year_code}"

    return url


def get_result_json(url, service_id):
    response = requests.get(url)
    result = response.content.decode()
    result_json = json.loads(result)
    return result_json[service_id]


def get_list_total_count(key, service_id, year_code=None) -> str:
    test_url = create_url(key, service_id, 1, 1, year_code)
    result_json = get_result_json(test_url, service_id)
    data = result_json["list_total_count"]

    return data


def get_data_list(key, service_id, start_index, end_index, year_code=None):
    url = create_url(key, service_id, start_index, end_index, year_code)
    data_list = get_result_json(url, service_id)
    row = data_list["row"]
    return row


def get_all_data_list(key, service_id, batch_size=999):
    df = pd.DataFrame()
    list_total_count = get_list_total_count(key, service_id)
    for start_index in range(1, list_total_count, batch_size):
        end_index = min(start_index + batch_size, list_total_count)
        data_list = get_data_list(key, service_id, start_index, end_index)

        list_df = pd.DataFrame(data_list)
        df = pd.concat([df, list_df])
    return df


def get_data_list_by_year(key, service_id, year_code, batch_size=999):
    df = pd.DataFrame()
    list_total_count = get_list_total_count(key, service_id, year_code)
    for start_index in range(1, list_total_count, batch_size):
        end_index = min(start_index + batch_size, list_total_count)
        data_list = get_data_list(key, service_id, start_index, end_index, year_code)

        list_df = pd.DataFrame(data_list)
        df = pd.concat([df, list_df])

    return df

# 행정동 단위 서울 생활인구

- 내국인 url: https://data.seoul.go.kr/dataList/OA-14991/S/1/datasetView.do#
- 장기외국인 url: https://data.seoul.go.kr/dataList/OA-14992/S/1/datasetView.do#
- 단기외국인 url: https://data.seoul.go.kr/dataList/OA-14993/S/1/datasetView.do#

API에 오류가 있어서 20251101 이전 데이터를 못 불러와서 파일로 처리해야함.

In [ ]:

import os
import dotenv

dotenv.load_dotenv()

# .env 파일에 서울 열린데이터 광장 API 키를 넣어주세요
SEOUL_DATA_OPEN_API_KEY = os.getenv("SEOUL_DATA_OPEN_API_KEY")
START_INDEX = 1
END_INDEX = 1

TMZON_PD_SE = "" # 시간대구분
ADSTRD_CODE_SE = "" # 행정동코드
SERVICE_ID = "SPOP_LOCAL_RESD_DONG"

output_dir = "./output"
output_file = "서울시_상권분석서비스_생활인구_행정동_2023_2025.csv"
output_path = os.path.join(output_dir, output_file)

column_mapping = {
    "STDR_DE_ID": "기준일ID",
    "TMZON_PD_SE": "시간대구분",
    "ADSTRD_CODE_SE": "행정동코드",
    "TOT_LVPOP_CO": "총생활인구수",
    
    # 남성 연령별
    "MALE_F0T9_LVPOP_CO": "남자0세부터9세생활인구수",
    "MALE_F10T14_LVPOP_CO": "남자10세부터14세생활인구수",
    "MALE_F15T19_LVPOP_CO": "남자15세부터19세생활인구수",
    "MALE_F20T24_LVPOP_CO": "남자20세부터24세생활인구수",
    "MALE_F25T29_LVPOP_CO": "남자25세부터29세생활인구수",
    "MALE_F30T34_LVPOP_CO": "남자30세부터34세생활인구수",
    "MALE_F35T39_LVPOP_CO": "남자35세부터39세생활인구수",
    "MALE_F40T44_LVPOP_CO": "남자40세부터44세생활인구수",
    "MALE_F45T49_LVPOP_CO": "남자45세부터49세생활인구수",
    "MALE_F50T54_LVPOP_CO": "남자50세부터54세생활인구수",
    "MALE_F55T59_LVPOP_CO": "남자55세부터59세생활인구수",
    "MALE_F60T64_LVPOP_CO": "남자60세부터64세생활인구수",
    "MALE_F65T69_LVPOP_CO": "남자65세부터69세생활인구수",
    "MALE_F70T74_LVPOP_CO": "남자70세이상생활인구수",
    
    # 여성 연령별
    "FEMALE_F0T9_LVPOP_CO": "여자0세부터9세생활인구수",
    "FEMALE_F10T14_LVPOP_CO": "여자10세부터14세생활인구수",
    "FEMALE_F15T19_LVPOP_CO": "여자15세부터19세생활인구수",
    "FEMALE_F20T24_LVPOP_CO": "여자20세부터24세생활인구수",
    "FEMALE_F25T29_LVPOP_CO": "여자25세부터29세생활인구수",
    "FEMALE_F30T34_LVPOP_CO": "여자30세부터34세생활인구수",
    "FEMALE_F35T39_LVPOP_CO": "여자35세부터39세생활인구수",
    "FEMALE_F40T44_LVPOP_CO": "여자40세부터44세생활인구수",
    "FEMALE_F45T49_LVPOP_CO": "여자45세부터49세생활인구수",
    "FEMALE_F50T54_LVPOP_CO": "여자50세부터54세생활인구수",
    "FEMALE_F55T59_LVPOP_CO": "여자55세부터59세생활인구수",
    "FEMALE_F60T64_LVPOP_CO": "여자60세부터64세생활인구수",
    "FEMALE_F65T69_LVPOP_CO": "여자65세부터69세생활인구수",
    "FEMALE_F70T74_LVPOP_CO": "여자70세이상생활인구수"
}

STDR_DE_ID = "20200401" # 기준일
url = create_api_url(
    SEOUL_DATA_OPEN_API_KEY,
    SERVICE_ID,
    START_INDEX,
    END_INDEX,
    STDR_DE_ID,
    TMZON_PD_SE,
    ADSTRD_CODE_SE
)

result = get_result_json(url, SERVICE_ID)

---

# 서울시 상권분석서비스 길단위인구 행정동

- URL: https://data.seoul.go.kr/dataList/OA-22178/S/1/datasetView.do

In [ ]:
import os
import dotenv

import pandas as pd


dotenv.load_dotenv()

# .env 파일에 서울 열린데이터 광장 API 키를 넣어주세요
SEOUL_DATA_OPEN_API_KEY = os.getenv("SEOUL_DATA_OPEN_API_KEY")
START_INDEX = 1
END_INDEX = 1000
SERVICE_ID = "VwsmAdstrdFlpopW"

output_dir = "../output"
output_file = "서울시_상권분석서비스_길단위인구_행정동_2019_2025.csv"
output_path = os.path.join(output_dir, output_file)


column_mapping = {
    "STDR_YYQU_CD": "기준_년분기_코드",
    "ADSTRD_CD": "행정동_코드",
    "ADSTRD_CD_NM": "행정동_코드_명",
    "TOT_FLPOP_CO": "총_유동인구_수",
    "ML_FLPOP_CO": "남성_유동인구_수",
    "FML_FLPOP_CO": "여성_유동인구_수",
    "AGRDE_10_FLPOP_CO": "연령대_10_유동인구_수",
    "AGRDE_20_FLPOP_CO": "연령대_20_유동인구_수",
    "AGRDE_30_FLPOP_CO": "연령대_30_유동인구_수",
    "AGRDE_40_FLPOP_CO": "연령대_40_유동인구_수",
    "AGRDE_50_FLPOP_CO": "연령대_50_유동인구_수",
    "AGRDE_60_ABOVE_FLPOP_CO": "연령대_60_이상_유동인구_수",
    "TMZON_00_06_FLPOP_CO": "시간대_00_06_유동인구_수",
    "TMZON_06_11_FLPOP_CO": "시간대_06_11_유동인구_수",
    "TMZON_11_14_FLPOP_CO": "시간대_11_14_유동인구_수",
    "TMZON_14_17_FLPOP_CO": "시간대_14_17_유동인구_수",
    "TMZON_17_21_FLPOP_CO": "시간대_17_21_유동인구_수",
    "TMZON_21_24_FLPOP_CO": "시간대_21_24_유동인구_수",
    "MON_FLPOP_CO": "월요일_유동인구_수",
    "TUES_FLPOP_CO": "화요일_유동인구_수",
    "WED_FLPOP_CO": "수요일_유동인구_수",
    "THUR_FLPOP_CO": "목요일_유동인구_수",
    "FRI_FLPOP_CO": "금요일_유동인구_수",
    "SAT_FLPOP_CO": "토요일_유동인구_수",
    "SUN_FLPOP_CO": "일요일_유동인구_수",
}

df = pd.DataFrame()

df = get_all_data_list(SEOUL_DATA_OPEN_API_KEY, SERVICE_ID)
df = df.rename(columns=column_mapping)
df = df.sort_values(by="기준_년분기_코드")
df.to_csv(output_path, index=False)

---


## 상권분석서비스: 직장인구
Source: 서울시_상권분석서비스_직장인구_행정동.ipynb

# 서울시 상권분석서비스 직장인구 행정동

- URL: https://data.seoul.go.kr/dataList/OA-22184/S/1/datasetView.do

In [ ]:
import os
import dotenv
import requests
import pandas as pd


dotenv.load_dotenv()

# .env 파일에 서울 열린데이터 광장 API 키를 넣어주세요
SEOUL_DATA_OPEN_API_KEY = os.getenv("SEOUL_DATA_OPEN_API_KEY")
START_INDEX = 1
END_INDEX = 1000
STDR_YYQU_CD_LIST = [
    "20231",
    "20232",
    "20233",
    "20234",
    "20241",
    "20242",
    "20243",
    "20244",
    "20251",
    "20252",
    "20253",
]

output_dir = "../output"
output_file = "서울시_상권분석서비스_직장인구_행정동_2023_2025.csv"
output_path = os.path.join(output_dir, output_file)


header_mapping = {
    "STDR_YYQU_CD": "기준_년분기_코드",
    "ADSTRD_CD": "행정동_코드",
    "ADSTRD_CD_NM": "행정동_코드_명",
    "TOT_WRC_POPLTN_CO": "총_직장_인구_수",
    "ML_WRC_POPLTN_CO": "남성_직장_인구_수",
    "FML_WRC_POPLTN_CO": "여성_직장_인구_수",
    "AGRDE_10_WRC_POPLTN_CO": "연령대_10_직장_인구_수",
    "AGRDE_20_WRC_POPLTN_CO": "연령대_20_직장_인구_수",
    "AGRDE_30_WRC_POPLTN_CO": "연령대_30_직장_인구_수",
    "AGRDE_40_WRC_POPLTN_CO": "연령대_40_직장_인구_수",
    "AGRDE_50_WRC_POPLTN_CO": "연령대_50_직장_인구_수",
    "AGRDE_60_ABOVE_WRC_POPLTN_CO": "연령대_60_이상_직장_인구_수",
    "MAG_10_WRC_POPLTN_CO": "남성연령대_10_직장_인구_수",
    "MAG_20_WRC_POPLTN_CO": "남성연령대_20_직장_인구_수",
    "MAG_30_WRC_POPLTN_CO": "남성연령대_30_직장_인구_수",
    "MAG_40_WRC_POPLTN_CO": "남성연령대_40_직장_인구_수",
    "MAG_50_WRC_POPLTN_CO": "남성연령대_50_직장_인구_수",
    "MAG_60_ABOVE_WRC_POPLTN_CO": "남성연령대_60_이상_직장_인구_수",
    "FAG_10_WRC_POPLTN_CO": "여성연령대_10_직장_인구_수",
    "FAG_20_WRC_POPLTN_CO": "여성연령대_20_직장_인구_수",
    "FAG_30_WRC_POPLTN_CO": "여성연령대_30_직장_인구_수",
    "FAG_40_WRC_POPLTN_CO": "여성연령대_40_직장_인구_수",
    "FAG_50_WRC_POPLTN_CO": "여성연령대_50_직장_인구_수",
    "FAG_60_ABOVE_WRC_POPLTN_CO": "여성연령대_60_이상_직장_인구_수",
}

df = pd.DataFrame()


for stdr_yyqu_cd in STDR_YYQU_CD_LIST:
    request_url = f"http://openapi.seoul.go.kr:8088/{SEOUL_DATA_OPEN_API_KEY}/json/VwsmAdstrdWrcPopltnW/{START_INDEX}/{END_INDEX}/{stdr_yyqu_cd}"
    response = requests.get(request_url)
    data = response.json()
    result = data["VwsmAdstrdWrcPopltnW"]

    new_df = pd.DataFrame(result["row"])
    new_df = new_df.rename(columns=header_mapping)

    df = pd.concat([df, new_df], ignore_index=True)
    print("Downloaded: ", stdr_yyqu_cd)

df.to_csv(output_path, index=False)

---


## 상권분석서비스: 추정매출
Source: 서울시_상권분석서비스_추정매출_행정동.ipynb

# 서울시 상권분석서비스 추정매출 행정동

- URL: https://data.seoul.go.kr/dataList/OA-22175/S/1/datasetView.do

In [ ]:
import os
import dotenv
import pandas as pd


dotenv.load_dotenv()


# .env 파일에 서울 열린데이터 광장 API 키를 넣어주세요
SEOUL_DATA_OPEN_API_KEY = os.getenv("SEOUL_DATA_OPEN_API_KEY")
START_INDEX = 1
END_INDEX = 1000
SERVICE_ID = "VwsmAdstrdSelngW"
STDR_YYQU_CD_LIST = [
    "20231",
    "20232",
    "20233",
    "20234",
    "20241",
    "20242",
    "20243",
    "20244",
    "20251",
    "20252",
    "20253",
]
output_dir = "../output"
output_file = "서울시_상권분석서비스_추정매출_행정동_2023_2025.csv"
output_path = os.path.join(output_dir, output_file)

column_mapping = {
    "STDR_YYQU_CD": "기준_년분기_코드",
    "ADSTRD_CD": "행정동_코드",
    "ADSTRD_CD_NM": "행정동_코드_명",
    "SVC_INDUTY_CD": "서비스_업종_코드",
    "SVC_INDUTY_CD_NM": "서비스_업종_코드_명",
    "THSMON_SELNG_AMT": "당월_매출_금액",
    "THSMON_SELNG_CO": "당월_매출_건수",
    "MDWK_SELNG_AMT": "주중_매출_금액",
    "WKEND_SELNG_AMT": "주말_매출_금액",
    "MON_SELNG_AMT": "월요일_매출_금액",
    "TUES_SELNG_AMT": "화요일_매출_금액",
    "WED_SELNG_AMT": "수요일_매출_금액",
    "THUR_SELNG_AMT": "목요일_매출_금액",
    "FRI_SELNG_AMT": "금요일_매출_금액",
    "SAT_SELNG_AMT": "토요일_매출_금액",
    "SUN_SELNG_AMT": "일요일_매출_금액",
    "TMZON_00_06_SELNG_AMT": "시간대_00~06_매출_금액",
    "TMZON_06_11_SELNG_AMT": "시간대_06~11_매출_금액",
    "TMZON_11_14_SELNG_AMT": "시간대_11~14_매출_금액",
    "TMZON_14_17_SELNG_AMT": "시간대_14~17_매출_금액",
    "TMZON_17_21_SELNG_AMT": "시간대_17~21_매출_금액",
    "TMZON_21_24_SELNG_AMT": "시간대_21~24_매출_금액",
    "ML_SELNG_AMT": "남성_매출_금액",
    "FML_SELNG_AMT": "여성_매출_금액",
    "AGRDE_10_SELNG_AMT": "연령대_10_매출_금액",
    "AGRDE_20_SELNG_AMT": "연령대_20_매출_금액",
    "AGRDE_30_SELNG_AMT": "연령대_30_매출_금액",
    "AGRDE_40_SELNG_AMT": "연령대_40_매출_금액",
    "AGRDE_50_SELNG_AMT": "연령대_50_매출_금액",
    "AGRDE_60_ABOVE_SELNG_AMT": "연령대_60_이상_매출_금액",
    "MDWK_SELNG_CO": "주중_매출_건수",
    "WKEND_SELNG_CO": "주말_매출_건수",
    "MON_SELNG_CO": "월요일_매출_건수",
    "TUES_SELNG_CO": "화요일_매출_건수",
    "WED_SELNG_CO": "수요일_매출_건수",
    "THUR_SELNG_CO": "목요일_매출_건수",
    "FRI_SELNG_CO": "금요일_매출_건수",
    "SAT_SELNG_CO": "토요일_매출_건수",
    "SUN_SELNG_CO": "일요일_매출_건수",
    "TMZON_00_06_SELNG_CO": "시간대_00~06_매출_건수",
    "TMZON_06_11_SELNG_CO": "시간대_06~11_매출_건수",
    "TMZON_11_14_SELNG_CO": "시간대_11~14_매출_건수",
    "TMZON_14_17_SELNG_CO": "시간대_14~17_매출_건수",
    "TMZON_17_21_SELNG_CO": "시간대_17~21_매출_건수",
    "TMZON_21_24_SELNG_CO": "시간대_21~24_매출_건수",
    "ML_SELNG_CO": "남성_매출_건수",
    "FML_SELNG_CO": "여성_매출_건수",
    "AGRDE_10_SELNG_CO": "연령대_10_매출_건수",
    "AGRDE_20_SELNG_CO": "연령대_20_매출_건수",
    "AGRDE_30_SELNG_CO": "연령대_30_매출_건수",
    "AGRDE_40_SELNG_CO": "연령대_40_매출_건수",
    "AGRDE_50_SELNG_CO": "연령대_50_매출_건수",
    "AGRDE_60_ABOVE_SELNG_CO": "연령대_60_이상_매출_건수",
}


df = pd.DataFrame()

get_list_total_count(SEOUL_DATA_OPEN_API_KEY, SERVICE_ID)

for stdr_yyqu_cd in STDR_YYQU_CD_LIST:
    new_df = get_data_list_by_year(SEOUL_DATA_OPEN_API_KEY, SERVICE_ID, stdr_yyqu_cd)
    new_df = new_df.rename(columns=column_mapping)
    df = pd.concat([df, new_df], ignore_index=True)
    print("Downloaded: ", stdr_yyqu_cd)

df.to_csv(output_path, index=False)

---